In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.multitest import multipletests
from tqdm import tqdm

/Users/tianji/mambaforge/envs/myenv/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Read in txt files and modify the column names to distinguish each other

In [2]:
def subset_df_and_rename_columns(fisher_test_input_df, suffix, glm_test_input_df=None, 
                                 add_unique_id=False, FDR=0.05, dI=0.1, coverage=20):
    """
    Processes Fisher test results and optionally adds GLM test results to create a comprehensive output DataFrame.
    
    Parameters:
    -----------
    fisher_test_input_df : pd.DataFrame
        Input DataFrame containing Fisher test results. Must include columns from 'gene' to 'FDR'.
        
    suffix : str
        Suffix to append to Fisher test columns (e.g., "(suffix)").
        
    glm_test_input_df : pd.DataFrame, optional
        Input DataFrame containing GLM test results. If provided, adds:
        - glm.pvalue: GLM p-values
        - glm.FDR: GLM false discovery rates
        - glm.filter: GLM significance filter (-1=repressed, 0=insignificant, 1=activated)
        
    add_unique_id : bool, default False
        If True, adds a 'uniqueID' column combining 'name' and 'isoformIDs'.
        
    FDR : float, default 0.05
        False Discovery Rate threshold for significance.
        
    dI : float, default 0.1
        Minimum absolute delta inclusion threshold for activation/repression.
        
    coverage : int, default 20
        Minimum coverage threshold for considering events.
    
    Returns:
    --------
    pd.DataFrame
        Processed DataFrame with genomic features and Fisher test results. If GLM input is provided,
        adds glm.pvalue, glm.FDR, and glm.filter columns.
    """
    # Process Fisher test DataFrame
    df_fisher = fisher_test_input_df.copy()
    df_fisher = df_fisher.loc[:, 'gene':'FDR'].copy()
    
    # Add chromLength column after chromEnd
    chrom_end_idx = df_fisher.columns.get_loc('chromEnd')
    df_fisher.insert(chrom_end_idx + 1, 'chromLength', 
                     df_fisher['chromEnd'] - df_fisher['chromStart'])
    
    # Create filter column for Fisher results
    df_fisher['filter'] = np.nan
    df_fisher.loc[df_fisher['coverage'] > coverage, 'filter'] = 0
    df_fisher.loc[
        (df_fisher['coverage'] > coverage) & 
        (df_fisher['dI_g1_vs_g2'] < -dI) & 
        (df_fisher['FDR'] < FDR),
        'filter'
    ] = -1
    df_fisher.loc[
        (df_fisher['coverage'] > coverage) & 
        (df_fisher['dI_g1_vs_g2'] > dI) & 
        (df_fisher['FDR'] < FDR),
        'filter'
    ] = 1
    
    # Rename Fisher columns with suffix
    df_fisher.rename(columns={
        'coverage': f'coverage({suffix})',
        'dI_g1_vs_g2': f'dI_g1_vs_g2({suffix})',
        'pvalue': f'pvalue({suffix})',
        'FDR': f'FDR({suffix})',
        'filter': f'filter({suffix})'
    }, inplace=True)
    
    # Add unique ID if requested
    if add_unique_id:
        isoform_idx = df_fisher.columns.get_loc('isoformIDs')
        df_fisher.insert(isoform_idx + 1, 'uniqueID', 
                         df_fisher['name'] + '_' + df_fisher['isoformIDs'])
    
    # Process GLM test DataFrame if provided
    if glm_test_input_df is not None:
        df_glm = glm_test_input_df.copy()
        # Create GLM filter column
        df_glm['glm.filter'] = np.nan
        df_glm.loc[df_glm['coverage'] > coverage, 'glm.filter'] = 0
        df_glm.loc[
            (df_glm['coverage'] > coverage) & 
            (df_glm['dI_g1_vs_g2'] < -dI) & 
            (df_glm['FDR'] < FDR),
            'glm.filter'
        ] = -1
        df_glm.loc[
            (df_glm['coverage'] > coverage) & 
            (df_glm['dI_g1_vs_g2'] > dI) & 
            (df_glm['FDR'] < FDR),
            'glm.filter'
        ] = 1
        
        # Create merge key based on whether uniqueID exists
        if add_unique_id and 'uniqueID' in df_fisher.columns:
            # Create matching uniqueID in GLM DataFrame
            df_glm['uniqueID'] = df_glm['name'] + '_' + df_glm['isoformIDs']
            merge_key = 'uniqueID'
        else:
            # Use genomic coordinates as merge key
            merge_key = ['gene', 'name', 'chrom', 'strand', 'chromStart', 'chromEnd', 'isoformIDs']
        
        # Select only the three GLM columns to merge
        glm_cols = df_glm[[merge_key] + ['pvalue', 'FDR', 'glm.filter']] \
            if isinstance(merge_key, str) \
            else df_glm[merge_key + ['pvalue', 'FDR', 'glm.filter']]
        
        # Rename GLM columns
        glm_cols = glm_cols.rename(columns={
            'pvalue': f'glm.pvalue({suffix})',
            'FDR': f'glm.FDR({suffix})',
            'glm.filter': f'glm.filter({suffix})'
        })
        
        # Merge with Fisher results
        df_fisher = pd.merge(
            df_fisher,
            glm_cols,
            on=merge_key,
            how='left'
        )
    
    return df_fisher

############### Cassette Exon ###############
cass_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Cass.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
cass_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Cass.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
cass_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Cass.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
cass_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Cass.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
cass_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Cass.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_fisher = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_glm = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/glm_test_results/HCT116.Cass.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_fisher = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_glm = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/glm_test_results/HCT116.Cass.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_Ctrl_IAA_50PctKu_flox_df_fisher = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_Ctrl_IAA_50PctKu_flox_df_glm = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/glm_test_results/HCT116.Cass.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_50PctKu_flox_0PctKu_flox_df_fisher = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_50PctKu_flox_0PctKu_flox_df_glm = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/glm_test_results/HCT116.Cass.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_fisher = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_glm = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Cass.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')

cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_fisher = cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_fisher.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_fisher = cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_fisher.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
cass_HCT116_Ctrl_IAA_50PctKu_flox_df_fisher = cass_HCT116_Ctrl_IAA_50PctKu_flox_df_fisher.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
cass_HCT116_50PctKu_flox_0PctKu_flox_df_fisher = cass_HCT116_50PctKu_flox_0PctKu_flox_df_fisher.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_fisher = cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_fisher.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})


cass_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(cass_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD')
cass_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(cass_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD')
cass_HEK293_KuKD_df = subset_df_and_rename_columns(cass_HEK293_KuKD_df, 'HEK293.KuKD')
cass_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(cass_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD')
cass_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(cass_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD')
    # Add GeneScript Samples
cass_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(fisher_test_input_df = cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_fisher, suffix='HCT116.100%Ku_vs_0%Ku', glm_test_input_df = cass_HCT116_Ctrl_IAA_0PctKu_IAA_df_glm)
cass_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(fisher_test_input_df = cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_fisher, suffix='HCT116.100%Ku_vs_10%Ku', glm_test_input_df = cass_HCT116_Ctrl_IAA_10PctKu_IAA_df_glm)
cass_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(fisher_test_input_df = cass_HCT116_Ctrl_IAA_50PctKu_flox_df_fisher, suffix='HCT116.100%Ku_vs_50%Ku', glm_test_input_df = cass_HCT116_Ctrl_IAA_50PctKu_flox_df_glm)
cass_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(fisher_test_input_df = cass_HCT116_50PctKu_flox_0PctKu_flox_df_fisher, suffix='HCT116.50%Ku_vs_0%Ku', glm_test_input_df = cass_HCT116_50PctKu_flox_0PctKu_flox_df_glm)
cass_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(fisher_test_input_df = cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_fisher, suffix='HCT116.10%Ku_vs_0%Ku', glm_test_input_df = cass_HCT116_10PctKu_IAA_0PctKu_IAA_df_glm)

############### Alternative 3' Splice Site ###############
alt3_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Alt3.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt3_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Alt3.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt3_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Alt3.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt3_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Alt3.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
alt3_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Alt3.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt3.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt3.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt3_HCT116_Ctrl_IAA_50PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt3.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
alt3_HCT116_50PctKu_flox_0PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt3.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt3.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df = alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df = alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
alt3_HCT116_Ctrl_IAA_50PctKu_flox_df = alt3_HCT116_Ctrl_IAA_50PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
alt3_HCT116_50PctKu_flox_0PctKu_flox_df = alt3_HCT116_50PctKu_flox_0PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df = alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})

alt3_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(alt3_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD', add_unique_id = True)
alt3_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(alt3_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD', add_unique_id = True)
alt3_HEK293_KuKD_df = subset_df_and_rename_columns(alt3_HEK293_KuKD_df, 'HEK293.KuKD', add_unique_id = True)
alt3_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(alt3_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD', add_unique_id = True)
alt3_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(alt3_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD', add_unique_id = True)
    # Add GeneScript Samples
alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df, 'HCT116.100%Ku_vs_0%Ku', add_unique_id = True)
alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df, 'HCT116.100%Ku_vs_10%Ku', add_unique_id = True)
alt3_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(alt3_HCT116_Ctrl_IAA_50PctKu_flox_df, 'HCT116.100%Ku_vs_50%Ku', add_unique_id = True)
alt3_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(alt3_HCT116_50PctKu_flox_0PctKu_flox_df, 'HCT116.50%Ku_vs_0%Ku', add_unique_id = True)
alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df, 'HCT116.10%Ku_vs_0%Ku', add_unique_id = True)

############### Alternative 5' Splice Site ###############
alt5_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Alt5.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt5_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Alt5.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt5_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Alt5.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
alt5_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Alt5.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
alt5_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Alt5.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt5.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt5.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt5_HCT116_Ctrl_IAA_50PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt5.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
alt5_HCT116_50PctKu_flox_0PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt5.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Alt5.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df = alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df = alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
alt5_HCT116_Ctrl_IAA_50PctKu_flox_df = alt5_HCT116_Ctrl_IAA_50PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
alt5_HCT116_50PctKu_flox_0PctKu_flox_df = alt5_HCT116_50PctKu_flox_0PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df = alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})

alt5_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(alt5_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD', add_unique_id=True)
alt5_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(alt5_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD', add_unique_id=True)
alt5_HEK293_KuKD_df = subset_df_and_rename_columns(alt5_HEK293_KuKD_df, 'HEK293.KuKD', add_unique_id=True)
alt5_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(alt5_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD', add_unique_id=True)
alt5_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(alt5_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD', add_unique_id=True)
    # Add GeneScript Samples
alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df, 'HCT116.100%Ku_vs_0%Ku', add_unique_id = True)
alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df, 'HCT116.100%Ku_vs_10%Ku', add_unique_id = True)
alt5_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(alt5_HCT116_Ctrl_IAA_50PctKu_flox_df, 'HCT116.100%Ku_vs_50%Ku', add_unique_id = True)
alt5_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(alt5_HCT116_50PctKu_flox_0PctKu_flox_df, 'HCT116.50%Ku_vs_0%Ku', add_unique_id = True)
alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df, 'HCT116.10%Ku_vs_0%Ku', add_unique_id = True)

############### Intron Retention ###############
iret_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Iret.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
iret_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Iret.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
iret_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Iret.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
iret_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Iret.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
iret_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Iret.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
iret_HCT116_Ctrl_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Iret.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
iret_HCT116_Ctrl_IAA_10PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Iret.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
iret_HCT116_Ctrl_IAA_50PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Iret.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
iret_HCT116_50PctKu_flox_0PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Iret.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
iret_HCT116_10PctKu_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Iret.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')

iret_HCT116_Ctrl_IAA_0PctKu_IAA_df = iret_HCT116_Ctrl_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
iret_HCT116_Ctrl_IAA_10PctKu_IAA_df = iret_HCT116_Ctrl_IAA_10PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
iret_HCT116_Ctrl_IAA_50PctKu_flox_df = iret_HCT116_Ctrl_IAA_50PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
iret_HCT116_50PctKu_flox_0PctKu_flox_df = iret_HCT116_50PctKu_flox_0PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
iret_HCT116_10PctKu_IAA_0PctKu_IAA_df = iret_HCT116_10PctKu_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})

iret_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(iret_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD')
iret_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(iret_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD')
iret_HEK293_KuKD_df = subset_df_and_rename_columns(iret_HEK293_KuKD_df, 'HEK293.KuKD')
iret_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(iret_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD')
iret_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(iret_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD')
    # Add GeneScript Samples
iret_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(iret_HCT116_Ctrl_IAA_0PctKu_IAA_df, 'HCT116.100%Ku_vs_0%Ku')
iret_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(iret_HCT116_Ctrl_IAA_10PctKu_IAA_df, 'HCT116.100%Ku_vs_10%Ku')
iret_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(iret_HCT116_Ctrl_IAA_50PctKu_flox_df, 'HCT116.100%Ku_vs_50%Ku')
iret_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(iret_HCT116_50PctKu_flox_0PctKu_flox_df, 'HCT116.50%Ku_vs_0%Ku')
iret_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(iret_HCT116_10PctKu_IAA_0PctKu_IAA_df, 'HCT116.10%Ku_vs_0%Ku')

############### Mutually Exclusive Exons ###############
mutx_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Mutx.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
mutx_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Mutx.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
mutx_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Mutx.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
mutx_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Mutx.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
mutx_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Mutx.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Mutx.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Mutx.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
mutx_HCT116_Ctrl_IAA_50PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Mutx.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
mutx_HCT116_50PctKu_flox_0PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Mutx.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Mutx.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')

mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df = mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df = mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
mutx_HCT116_Ctrl_IAA_50PctKu_flox_df = mutx_HCT116_Ctrl_IAA_50PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
mutx_HCT116_50PctKu_flox_0PctKu_flox_df = mutx_HCT116_50PctKu_flox_0PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df = mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})

mutx_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(mutx_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD', add_unique_id=True)
mutx_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(mutx_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD', add_unique_id=True)
mutx_HEK293_KuKD_df = subset_df_and_rename_columns(mutx_HEK293_KuKD_df, 'HEK293.KuKD', add_unique_id=True)
mutx_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(mutx_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD', add_unique_id=True)
mutx_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(mutx_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD', add_unique_id=True)
    # Add GeneScript Samples
mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df, 'HCT116.100%Ku_vs_0%Ku', add_unique_id = True)
mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df, 'HCT116.100%Ku_vs_10%Ku', add_unique_id = True)
mutx_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(mutx_HCT116_Ctrl_IAA_50PctKu_flox_df, 'HCT116.100%Ku_vs_50%Ku', add_unique_id = True)
mutx_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(mutx_HCT116_50PctKu_flox_0PctKu_flox_df, 'HCT116.50%Ku_vs_0%Ku', add_unique_id = True)
mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df, 'HCT116.10%Ku_vs_0%Ku', add_unique_id = True)

############### Tandem Cassette Exons ###############
taca_cyc_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Taca.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
taca_g1arr_HCT116_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Taca.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
taca_HEK293_KuKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HEK293.Taca.Ctrl.KuKD.AlterSpliceResults.txt', delimiter='\t')
taca_cyc_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/Cyc.HCT116.Taca.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
taca_g1arr_HCT116_DNAPKcsKD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/G1Arr.HCT116.Taca.Ctrl.DNAPKcsKD.AlterSpliceResults.txt', delimiter='\t')
    # Add GeneScript Samples
taca_HCT116_Ctrl_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Taca.Ctrl_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
taca_HCT116_Ctrl_IAA_10PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Taca.Ctrl_IAA.10PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')
taca_HCT116_Ctrl_IAA_50PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Taca.Ctrl_IAA.50PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
taca_HCT116_50PctKu_flox_0PctKu_flox_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Taca.50PctKu_flox.0PctKu_flox.AlterSpliceResults.txt', delimiter='\t')
taca_HCT116_10PctKu_IAA_0PctKu_IAA_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/DSE_Results_Data/fisher_exact_test_results/HCT116.Taca.10PctKu_IAA.0PctKu_IAA.AlterSpliceResults.txt', delimiter='\t')

taca_HCT116_Ctrl_IAA_0PctKu_IAA_df = taca_HCT116_Ctrl_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.1)'
})
taca_HCT116_Ctrl_IAA_10PctKu_IAA_df = taca_HCT116_Ctrl_IAA_10PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.2)',
    'I_g2(KuKD)': 'I_g2(KuKD_10%Ku.1)'
})
taca_HCT116_Ctrl_IAA_50PctKu_flox_df = taca_HCT116_Ctrl_IAA_50PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_100%Ku.3)',
    'I_g2(KuKD)': 'I_g2(KuKD_50%Ku)'
})
taca_HCT116_50PctKu_flox_0PctKu_flox_df = taca_HCT116_50PctKu_flox_0PctKu_flox_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_50%KuFlox)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%KuFlox)'
})
taca_HCT116_10PctKu_IAA_0PctKu_IAA_df = taca_HCT116_10PctKu_IAA_0PctKu_IAA_df.rename(columns={
    'I_g1(Ctrl)': 'I_g1(Ctrl_10%Ku.1)',
    'I_g2(KuKD)': 'I_g2(KuKD_0%Ku.2)'
})

taca_cyc_HCT116_KuKD_df = subset_df_and_rename_columns(taca_cyc_HCT116_KuKD_df, 'Cyc.HCT116.KuKD')
taca_g1arr_HCT116_KuKD_df = subset_df_and_rename_columns(taca_g1arr_HCT116_KuKD_df, 'G1Arr.HCT116.KuKD')
taca_HEK293_KuKD_df = subset_df_and_rename_columns(taca_HEK293_KuKD_df, 'HEK293.KuKD')
taca_cyc_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(taca_cyc_HCT116_DNAPKcsKD_df, 'Cyc.HCT116.DNAPKcsKD')
taca_g1arr_HCT116_DNAPKcsKD_df = subset_df_and_rename_columns(taca_g1arr_HCT116_DNAPKcsKD_df, 'G1Arr.HCT116.DNAPKcsKD')
    # Add GeneScript Samples
taca_HCT116_Ctrl_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(taca_HCT116_Ctrl_IAA_0PctKu_IAA_df, 'HCT116.100%Ku_vs_0%Ku')
taca_HCT116_Ctrl_IAA_10PctKu_IAA_df = subset_df_and_rename_columns(taca_HCT116_Ctrl_IAA_10PctKu_IAA_df, 'HCT116.100%Ku_vs_10%Ku')
taca_HCT116_Ctrl_IAA_50PctKu_flox_df = subset_df_and_rename_columns(taca_HCT116_Ctrl_IAA_50PctKu_flox_df, 'HCT116.100%Ku_vs_50%Ku')
taca_HCT116_50PctKu_flox_0PctKu_flox_df = subset_df_and_rename_columns(taca_HCT116_50PctKu_flox_0PctKu_flox_df, 'HCT116.50%Ku_vs_0%Ku')
taca_HCT116_10PctKu_IAA_0PctKu_IAA_df = subset_df_and_rename_columns(taca_HCT116_10PctKu_IAA_0PctKu_IAA_df, 'HCT116.10%Ku_vs_0%Ku')

In [4]:
# List of all DataFrames and their labels
dfs = {
    "cass_cyc_HCT116_KuKD": cass_cyc_HCT116_KuKD_df,
    "cass_g1arr_HCT116_KuKD": cass_g1arr_HCT116_KuKD_df,
    "cass_HEK293_KuKD": cass_HEK293_KuKD_df,
    "cass_cyc_HCT116_DNAPKcsKD": cass_cyc_HCT116_DNAPKcsKD_df,
    "cass_g1arr_HCT116_DNAPKcsKD": cass_g1arr_HCT116_DNAPKcsKD_df,
    "cass_HCT116_100%Ku_vs_0%Ku": cass_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "cass_HCT116_100%Ku_vs_10%Ku": cass_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "cass_HCT116_100%Ku_vs_50%Ku": cass_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "cass_HCT116_50%Ku_vs_0%Ku": cass_HCT116_50PctKu_flox_0PctKu_flox_df,
    "cass_HCT116_10%Ku_vs_0%Ku": cass_HCT116_10PctKu_IAA_0PctKu_IAA_df,


    
    "alt3_cyc_HCT116_KuKD": alt3_cyc_HCT116_KuKD_df,
    "alt3_g1arr_HCT116_KuKD": alt3_g1arr_HCT116_KuKD_df,
    "alt3_HEK293_KuKD": alt3_HEK293_KuKD_df,
    "alt3_cyc_HCT116_DNAPKcsKD": alt3_cyc_HCT116_DNAPKcsKD_df,
    "alt3_g1arr_HCT116_DNAPKcsKD": alt3_g1arr_HCT116_DNAPKcsKD_df,
    "alt3_HCT116_100%Ku_vs_0%Ku": alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "alt3_HCT116_100%Ku_vs_10%Ku": alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "alt3_HCT116_100%Ku_vs_50%Ku": alt3_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "alt3_HCT116_50%Ku_vs_0%Ku": alt3_HCT116_50PctKu_flox_0PctKu_flox_df,
    "alt3_HCT116_10%Ku_vs_0%Ku": alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df,

    
    "alt5_cyc_HCT116_KuKD": alt5_cyc_HCT116_KuKD_df,
    "alt5_g1arr_HCT116_KuKD": alt5_g1arr_HCT116_KuKD_df,
    "alt5_HEK293_KuKD": alt5_HEK293_KuKD_df,
    "alt5_cyc_HCT116_DNAPKcsKD": alt5_cyc_HCT116_DNAPKcsKD_df,
    "alt5_g1arr_HCT116_DNAPKcsKD": alt5_g1arr_HCT116_DNAPKcsKD_df,
    "alt5_HCT116_100%Ku_vs_0%Ku": alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "alt5_HCT116_100%Ku_vs_10%Ku": alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "alt5_HCT116_100%Ku_vs_50%Ku": alt5_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "alt5_HCT116_50%Ku_vs_0%Ku": alt5_HCT116_50PctKu_flox_0PctKu_flox_df,
    "alt5_HCT116_10%Ku_vs_0%Ku": alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df,
    
    "iret_cyc_HCT116_KuKD": iret_cyc_HCT116_KuKD_df,
    "iret_g1arr_HCT116_KuKD": iret_g1arr_HCT116_KuKD_df,
    "iret_HEK293_KuKD": iret_HEK293_KuKD_df,
    "iret_cyc_HCT116_DNAPKcsKD": iret_cyc_HCT116_DNAPKcsKD_df,
    "iret_g1arr_HCT116_DNAPKcsKD": iret_g1arr_HCT116_DNAPKcsKD_df,
    "iret_HCT116_100%Ku_vs_0%Ku": iret_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "iret_HCT116_100%Ku_vs_10%Ku": iret_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "iret_HCT116_100%Ku_vs_50%Ku": iret_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "iret_HCT116_50%Ku_vs_0%Ku": iret_HCT116_50PctKu_flox_0PctKu_flox_df,
    "iret_HCT116_10%Ku_vs_0%Ku": iret_HCT116_10PctKu_IAA_0PctKu_IAA_df,
    
    "mutx_cyc_HCT116_KuKD": mutx_cyc_HCT116_KuKD_df,
    "mutx_g1arr_HCT116_KuKD": mutx_g1arr_HCT116_KuKD_df,
    "mutx_HEK293_KuKD": mutx_HEK293_KuKD_df,
    "mutx_cyc_HCT116_DNAPKcsKD": mutx_cyc_HCT116_DNAPKcsKD_df,
    "mutx_g1arr_HCT116_DNAPKcsKD": mutx_g1arr_HCT116_DNAPKcsKD_df,
    "mutx_HCT116_100%Ku_vs_0%Ku": mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "mutx_HCT116_100%Ku_vs_10%Ku": mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "mutx_HCT116_100%Ku_vs_50%Ku": mutx_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "mutx_HCT116_50%Ku_vs_0%Ku": mutx_HCT116_50PctKu_flox_0PctKu_flox_df,
    "mutx_HCT116_10%Ku_vs_0%Ku": mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df,
    
    "taca_cyc_HCT116_KuKD": taca_cyc_HCT116_KuKD_df,
    "taca_g1arr_HCT116_KuKD": taca_g1arr_HCT116_KuKD_df,
    "taca_HEK293_KuKD": taca_HEK293_KuKD_df,
    "taca_cyc_HCT116_DNAPKcsKD": taca_cyc_HCT116_DNAPKcsKD_df,
    "taca_g1arr_HCT116_DNAPKcsKD": taca_g1arr_HCT116_DNAPKcsKD_df,
    "taca_HCT116_100%Ku_vs_0%Ku": taca_HCT116_Ctrl_IAA_0PctKu_IAA_df,
    "taca_HCT116_100%Ku_vs_10%Ku": taca_HCT116_Ctrl_IAA_10PctKu_IAA_df,
    "taca_HCT116_100%Ku_vs_50%Ku": taca_HCT116_Ctrl_IAA_50PctKu_flox_df,
    "taca_HCT116_50%Ku_vs_0%Ku": taca_HCT116_50PctKu_flox_0PctKu_flox_df,
    "taca_HCT116_10%Ku_vs_0%Ku": taca_HCT116_10PctKu_IAA_0PctKu_IAA_df,
}


# Print the number of samples in each DataFrame
for name, df in dfs.items():
    print(f"{name}: {len(df)} samples")


cass_cyc_HCT116_KuKD: 42761 samples
cass_g1arr_HCT116_KuKD: 42761 samples
cass_HEK293_KuKD: 42761 samples
cass_cyc_HCT116_DNAPKcsKD: 42761 samples
cass_g1arr_HCT116_DNAPKcsKD: 42761 samples
cass_HCT116_100%Ku_vs_0%Ku: 42761 samples
cass_HCT116_100%Ku_vs_10%Ku: 42761 samples
cass_HCT116_100%Ku_vs_50%Ku: 42761 samples
cass_HCT116_50%Ku_vs_0%Ku: 42761 samples
cass_HCT116_10%Ku_vs_0%Ku: 42761 samples
alt3_cyc_HCT116_KuKD: 20617 samples
alt3_g1arr_HCT116_KuKD: 20617 samples
alt3_HEK293_KuKD: 20617 samples
alt3_cyc_HCT116_DNAPKcsKD: 20617 samples
alt3_g1arr_HCT116_DNAPKcsKD: 20617 samples
alt3_HCT116_100%Ku_vs_0%Ku: 20617 samples
alt3_HCT116_100%Ku_vs_10%Ku: 20617 samples
alt3_HCT116_100%Ku_vs_50%Ku: 20617 samples
alt3_HCT116_50%Ku_vs_0%Ku: 20617 samples
alt3_HCT116_10%Ku_vs_0%Ku: 20617 samples
alt5_cyc_HCT116_KuKD: 8066 samples
alt5_g1arr_HCT116_KuKD: 8066 samples
alt5_HEK293_KuKD: 8066 samples
alt5_cyc_HCT116_DNAPKcsKD: 8066 samples
alt5_g1arr_HCT116_DNAPKcsKD: 8066 samples
alt5_HCT116_100

### For each of the 6 types of AS, merge 5 different conditions and save the merged file

In [5]:
def merge_two_df(df, df2merge, df2merge_suffix, df2merge_Ig1_suffix, df2merge_Ig2_suffix, merge_on_unique_id=False):
    
    # Base columns to always merge
    base_columns = [
        f'coverage({df2merge_suffix})', 
        f'I_g1({df2merge_Ig1_suffix})', 
        f'I_g2({df2merge_Ig2_suffix})',
        f'dI_g1_vs_g2({df2merge_suffix})', 
        f'pvalue({df2merge_suffix})', 
        f'FDR({df2merge_suffix})',
        f'filter({df2merge_suffix})'
    ]
    
    # GLM columns to merge if they exist
    glm_columns = [
        f'glm.pvalue({df2merge_suffix})',
        f'glm.FDR({df2merge_suffix})',
        f'glm.filter({df2merge_suffix})'
    ]
    
    # Check which GLM columns actually exist in df2merge
    existing_glm_columns = [col for col in glm_columns if col in df2merge.columns]
    
    # Combine all columns to merge
    all_columns = base_columns + existing_glm_columns
    
    if not merge_on_unique_id:
        # Merge on 'name' column
        key_column = 'name'
        columns_to_select = [key_column] + all_columns
        df2merge_subset = df2merge[columns_to_select]
        merged_df = pd.merge(df, df2merge_subset, on=key_column, how='left')
    else:
        # Merge on 'uniqueID' column
        key_column = 'uniqueID'
        columns_to_select = [key_column] + all_columns
        df2merge_subset = df2merge[columns_to_select]
        merged_df = pd.merge(df, df2merge_subset, on=key_column, how='left')
    
    return merged_df

# Cassette Exon
cass_merged_df = merge_two_df(cass_cyc_HCT116_KuKD_df, cass_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD')
cass_merged_df = merge_two_df(cass_merged_df, cass_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD')
cass_merged_df = merge_two_df(cass_merged_df, cass_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD')
cass_merged_df = merge_two_df(cass_merged_df, cass_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD')
cass_merged_df = merge_two_df(cass_merged_df, cass_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1')
cass_merged_df = merge_two_df(cass_merged_df, cass_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1')
cass_merged_df = merge_two_df(cass_merged_df, cass_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku')
cass_merged_df = merge_two_df(cass_merged_df, cass_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox')
cass_merged_df = merge_two_df(cass_merged_df, cass_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2')

# Alternative 3' Splice Site
alt3_merged_df = merge_two_df(alt3_cyc_HCT116_KuKD_df, alt3_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox', merge_on_unique_id = True)
alt3_merged_df = merge_two_df(alt3_merged_df, alt3_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2', merge_on_unique_id = True)

# Alternative 5' Splice Site
alt5_merged_df = merge_two_df(alt5_cyc_HCT116_KuKD_df, alt5_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD', merge_on_unique_id=True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD', merge_on_unique_id=True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox', merge_on_unique_id = True)
alt5_merged_df = merge_two_df(alt5_merged_df, alt5_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2', merge_on_unique_id = True)

# Intron Retention
iret_merged_df = merge_two_df(iret_cyc_HCT116_KuKD_df, iret_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD')
iret_merged_df = merge_two_df(iret_merged_df, iret_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD')
iret_merged_df = merge_two_df(iret_merged_df, iret_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD')
iret_merged_df = merge_two_df(iret_merged_df, iret_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD')
iret_merged_df = merge_two_df(iret_merged_df, iret_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1')
iret_merged_df = merge_two_df(iret_merged_df, iret_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1')
iret_merged_df = merge_two_df(iret_merged_df, iret_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku')
iret_merged_df = merge_two_df(iret_merged_df, iret_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox')
iret_merged_df = merge_two_df(iret_merged_df, iret_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2')

# Mutually Exclusive Exons
mutx_merged_df = merge_two_df(mutx_cyc_HCT116_KuKD_df, mutx_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD', merge_on_unique_id=True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD', merge_on_unique_id=True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD', merge_on_unique_id=True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD', merge_on_unique_id=True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1', merge_on_unique_id = True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1', merge_on_unique_id = True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku', merge_on_unique_id = True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox', merge_on_unique_id = True)
mutx_merged_df = merge_two_df(mutx_merged_df, mutx_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2', merge_on_unique_id = True)

# Tandem Cassette Exons
taca_merged_df = merge_two_df(taca_cyc_HCT116_KuKD_df, taca_g1arr_HCT116_KuKD_df, df2merge_suffix = 'G1Arr.HCT116.KuKD', df2merge_Ig1_suffix = 'G1arr_Ctrl', df2merge_Ig2_suffix = 'G1arr_KuKD')
taca_merged_df = merge_two_df(taca_merged_df, taca_HEK293_KuKD_df, df2merge_suffix = 'HEK293.KuKD', df2merge_Ig1_suffix = 'Ctrl', df2merge_Ig2_suffix = 'KuKD')
taca_merged_df = merge_two_df(taca_merged_df, taca_cyc_HCT116_DNAPKcsKD_df, df2merge_suffix = 'Cyc.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'Cyc_Ctrl', df2merge_Ig2_suffix = 'Cyc_DNAPKcsKD')
taca_merged_df = merge_two_df(taca_merged_df, taca_g1arr_HCT116_DNAPKcsKD_df, df2merge_suffix = 'G1Arr.HCT116.DNAPKcsKD', df2merge_Ig1_suffix = 'G1Arr_Ctrl', df2merge_Ig2_suffix = 'G1Arr_DNAPKcsKD')
taca_merged_df = merge_two_df(taca_merged_df, taca_HCT116_Ctrl_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.1')
taca_merged_df = merge_two_df(taca_merged_df, taca_HCT116_Ctrl_IAA_10PctKu_IAA_df, df2merge_suffix = 'HCT116.100%Ku_vs_10%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.2', df2merge_Ig2_suffix = 'KuKD_10%Ku.1')
taca_merged_df = merge_two_df(taca_merged_df, taca_HCT116_Ctrl_IAA_50PctKu_flox_df, df2merge_suffix = 'HCT116.100%Ku_vs_50%Ku', df2merge_Ig1_suffix = 'Ctrl_100%Ku.3', df2merge_Ig2_suffix = 'KuKD_50%Ku')
taca_merged_df = merge_two_df(taca_merged_df, taca_HCT116_50PctKu_flox_0PctKu_flox_df, df2merge_suffix = 'HCT116.50%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_50%KuFlox', df2merge_Ig2_suffix = 'KuKD_0%KuFlox')
taca_merged_df = merge_two_df(taca_merged_df, taca_HCT116_10PctKu_IAA_0PctKu_IAA_df, df2merge_suffix = 'HCT116.10%Ku_vs_0%Ku', df2merge_Ig1_suffix = 'Ctrl_10%Ku.1', df2merge_Ig2_suffix = 'KuKD_0%Ku.2')

In [6]:
cass_merged_df.columns

Index(['gene', 'chrom', 'chromStart', 'chromEnd', 'chromLength', 'name',
       'score', 'strand', 'type', 'isoformIDs', 'coverage(Cyc.HCT116.KuKD)',
       'I_g1(Cyc_Ctrl)_x', 'I_g2(Cyc_KuKD)', 'dI_g1_vs_g2(Cyc.HCT116.KuKD)',
       'pvalue(Cyc.HCT116.KuKD)', 'FDR(Cyc.HCT116.KuKD)',
       'filter(Cyc.HCT116.KuKD)', 'coverage(G1Arr.HCT116.KuKD)',
       'I_g1(G1arr_Ctrl)', 'I_g2(G1arr_KuKD)',
       'dI_g1_vs_g2(G1Arr.HCT116.KuKD)', 'pvalue(G1Arr.HCT116.KuKD)',
       'FDR(G1Arr.HCT116.KuKD)', 'filter(G1Arr.HCT116.KuKD)',
       'coverage(HEK293.KuKD)', 'I_g1(Ctrl)', 'I_g2(KuKD)',
       'dI_g1_vs_g2(HEK293.KuKD)', 'pvalue(HEK293.KuKD)', 'FDR(HEK293.KuKD)',
       'filter(HEK293.KuKD)', 'coverage(Cyc.HCT116.DNAPKcsKD)',
       'I_g1(Cyc_Ctrl)_y', 'I_g2(Cyc_DNAPKcsKD)',
       'dI_g1_vs_g2(Cyc.HCT116.DNAPKcsKD)', 'pvalue(Cyc.HCT116.DNAPKcsKD)',
       'FDR(Cyc.HCT116.DNAPKcsKD)', 'filter(Cyc.HCT116.DNAPKcsKD)',
       'coverage(G1Arr.HCT116.DNAPKcsKD)', 'I_g1(G1Arr_Ctrl)',
       'I_g2

### Merge Gtex Data, Brain Studer Data, Studerr Data, NVD Data, and Alu tags to cass_merged_df

In [7]:
# Merge Tissue psi data downloaded from Gtex to the merged_df
gtex_file = '/Users/tianji/Desktop/Alu Project/General Data/GTEx.anno.txt'
gtex_df = pd.read_csv(gtex_file, sep='\t', comment='#')
gtex_df.drop('NAME', axis=1, inplace=True)
cass_df_with_gtex_psi = pd.merge(cass_merged_df, gtex_df, on='name', how='left')

In [8]:
# Merge NMD columns to data frames
NMD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/General Data/Exon Information/Exon_size_NMD_TYPE.info.csv')
# Select only the needed columns from NMD_df
nmd_subset = NMD_df[['name', 'NMD type', 'Ts NMD_ex', 'Ts NMD_in', 'total NMD']]

# Merge with cass_df_with_gtex_psi on 'name'
cass_df_with_gtex_psi = cass_df_with_gtex_psi.merge(
    nmd_subset,
    on='name',
    how='left'   # use 'inner' if you only want overlapping rows
)


/var/folders/gp/l_62hzx91kz0hng_60kps5t00000gn/T/ipykernel_62732/1156773686.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  NMD_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/General Data/Exon Information/Exon_size_NMD_TYPE.info.csv')


In [9]:
# Merge Brain Span and Studer data to cass_df_with_gtex_psi
studer_brainspan_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/analysis/Bryan.neuro.development.info.csv')
print(f'Brain span and Studer data has {len(studer_brainspan_df)} samples')
# Select columns from 'hPSC' to 'neuron_DIV100'
columns_1 = studer_brainspan_df.loc[:, 'hPSC':'neuron_DIV100'].columns.tolist()

# Select columns from '2B.10-12pcw' to '11.20+yr'
columns_2 = studer_brainspan_df.loc[:, '2B.10-12pcw':'11.20+yr'].columns.tolist()

# Combine the two sets of columns, including 'name' for merging
columns_to_merge = ['name'] + columns_1 + columns_2

# Merge the selected columns into df
cass_df_with_gtex_brainspan_studer_psi = cass_df_with_gtex_psi.merge(studer_brainspan_df[columns_to_merge], on='name', how='left')

# Display the merged DataFrame
print(f'Final Cassette merged df has {len(cass_df_with_gtex_brainspan_studer_psi)} samples')

Brain span and Studer data has 42761 samples
Final Cassette merged df has 42761 samples


### Merge Alu Tags to cass_df_with_gtex_brainspan_studer_psi

In [10]:
# Please refer to 
merged_ku_overlap_alu_df = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_Ku_Overlap_Alu/Merged_Ku_Overlap_Alu.csv')
cass_df_with_gtex_brainspan_studer_psi_alu = cass_df_with_gtex_brainspan_studer_psi.merge(merged_ku_overlap_alu_df, on='name', how='left')
print(len(cass_df_with_gtex_brainspan_studer_psi_alu))

42761


### Add a column "alu_type" to describe each exon event

In [11]:
"""First define different conditions for asAlu, irAlu, ssAlu and other cassette exons:
    --asAlu: 2nd exon contains asAlu
    --irAlu: (ui has asAlu and di has ssAlu OR ui has ssAlu and di has asAlu) AND (2nd exon has no asAlu)
    --ssAlu: 2nd exon contains ssAlu but not asAlu AND Not (ui has asAlu and di has ssAlu OR ui has ssAlu and di has asAlu)
    --other:
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

# Simplify the name of cass_df_with_gtex_brainspan_studer_psi_alu
cass_df = cass_df_with_gtex_brainspan_studer_psi_alu.copy()

# Condition for asAlu
asAlu_condition = (cass_df['all_asAlu_exon'] > 0.0)

# Two pre-conditions for irAlu
irAlu_pre_condition1 = ((cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0))
irAlu_pre_condition2 = ((cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0))
# Condition for irAlu
irAlu_condition = ((irAlu_pre_condition1 | irAlu_pre_condition2) & 
                                   (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0))

# Condition for ssAlu
ssAlu_condition = ((cass_df['all_ssAlu_exon'] > 0.0) & 
                   (cass_df['all_asAlu_exon'] == 0.0))

# Condition for other cassette exons
other_samples_condition = ~(asAlu_condition | irAlu_condition | ssAlu_condition)

# Check for overlapping categories
overlap_mask = (
    (asAlu_condition & irAlu_condition) |
    (asAlu_condition & ssAlu_condition) |
    (irAlu_condition & ssAlu_condition)
)
print(f"Number of samples in multiple categories: {overlap_mask.sum()}")

Number of samples in multiple categories: 0


In [12]:
"""Create a new column 'alu_type' after 'isoformIDs' column to annotate the type of each cassette exon event"""
# Create alu_type column using priority: asAlu > irAlu > ssAlu > other
conditions = [
    asAlu_condition,
    irAlu_condition,
    ssAlu_condition,
    other_samples_condition
]

choices = ['asAlu', 'irAlu', 'ssAlu', 'other']

# Insert new column after 'isoformIDs'
cass_df.insert(
    loc=cass_df.columns.get_loc('isoformIDs') + 1,
    column='alu_type',
    value=np.select(conditions[:-1], choices[:-1], default='other')
)

# Verify distribution
print("\nCategory counts:")
print(cass_df['alu_type'].value_counts())


Category counts:
alu_type
other    24652
irAlu    14991
asAlu     2678
ssAlu      440
Name: count, dtype: int64


### Add a column "alu_subtype" to describe each exon event

In [13]:
########################################################################
"""Define different conditions for asAlu subtypes"""
########################################################################
""" --(1) asAlu_not_flanked_by_irAlu: 2nd exon contains asAlu, but ui and di does not contain irAlu
    --asAlu_flanked_by_irAlu: 2nd exon contains asAlu and ui and di also contains irAlu pair
                                                            |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
        > (2) asAlu_flanked_by_irAlu_without_local_competition:          <=         <=        =>     (irAlu_pre_condition1)
                                                                    or   =>         <=        <=     (irAlu_pre_condition2)
                                                            --------------------------------------------------
        > (3) asAlu_flanked_by_irAlu_with_local_competition:           <=  =>       <=        =>     (subcondition1)
                                                                    or   <=         <=      =>  <=   (subcondition2)
                                                                    or =>  <=       <=        <=     (subcondition3)
                                                                    or   =>         <=      <=  =>   (subcondition4)                                                                                 
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

"""Now let's set up conditions to break asAlu type into 3 subtypes"""
# conditions for asAlu not flanked by irAlu
asAlu_not_flanked_by_irAlu_condition = ~(irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition

# Four types of Local Competing irAlu subconditions
subcondition1 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
subcondition2 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
subcondition3 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
subcondition4 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
# Local irAlu Competing conditions
local_irAlu_competing_conditions = subcondition1 | subcondition2 | subcondition3 | subcondition4

# conditions for asAlu flanked by non-local competing irAlu
asAlu_flanked_by_irAlu_without_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition & ~local_irAlu_competing_conditions

# conditions for asAlu flanked by local competing irAlu
asAlu_flanked_by_irAlu_with_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & asAlu_condition & local_irAlu_competing_conditions

# First verify the conditions
total_asAlu = asAlu_condition.sum()
sum_subtypes = (asAlu_not_flanked_by_irAlu_condition | 
               asAlu_flanked_by_irAlu_without_local_competition |
               asAlu_flanked_by_irAlu_with_local_competition).sum()

condition1_met = (sum_subtypes == total_asAlu)
if condition1_met:
    print("Great! Sum of three asAlu subtypes equal to total asAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of three asAlu subtypes does not equal to total asAlu")

# Check for overlaps between subtypes
overlap1 = (asAlu_not_flanked_by_irAlu_condition & 
           asAlu_flanked_by_irAlu_without_local_competition).sum()
overlap2 = (asAlu_not_flanked_by_irAlu_condition & 
           asAlu_flanked_by_irAlu_with_local_competition).sum()
overlap3 = (asAlu_flanked_by_irAlu_without_local_competition & 
           asAlu_flanked_by_irAlu_with_local_competition).sum()
condition2_met = (overlap1 + overlap2 + overlap3) == 0
if condition2_met:
    print(f"Great! No overlapping samples between asAlu subtypes")
else:
    print(f"Oh no! Overlapping samples between asAlu subtypes are found")

"""Define different conditions for ssAlu subtypes:
    --(1) ssAlu_not_flanked_by_irAlu: 2nd exon contains ssAlu, but ui and di does not contain irAlu
    --ssAlu_flanked_by_irAlu: 2nd exon contains ssAlu and ui and di also contains irAlu pair
                                                            |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
        > (2) ssAlu_flanked_by_irAlu_without_local_competition:          <=         =>        =>     (irAlu_pre_condition1)
                                                                    or   =>         =>        <=     (irAlu_pre_condition2)
                                                            --------------------------------------------------
        > (3) ssAlu_flanked_by_irAlu_with_local_competition:           <=  =>       =>        =>     (subcondition1)
                                                                    or   <=         =>      =>  <=   (subcondition2)
                                                                    or =>  <=       =>        <=     (subcondition3)
                                                                    or   =>         =>      <=  =>   (subcondition4)                                                                                 
    (Notation) (1) ui: upstream intron  (2) di: downstream intron
    """

########################################################################
"""Now let's set up conditions to break ssAlu type into 3 subtypes"""
########################################################################
# conditions for asAlu not flanked by irAlu
ssAlu_not_flanked_by_irAlu_condition = ~(irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition

# Four types of Local Competing irAlu subconditions
subcondition1 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
subcondition2 = irAlu_pre_condition1 & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
subcondition3 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
subcondition4 = irAlu_pre_condition2 & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
# Local irAlu Competing conditions
local_irAlu_competing_conditions = subcondition1 | subcondition2 | subcondition3 | subcondition4

# conditions for asAlu flanked by non-local competing irAlu
ssAlu_flanked_by_irAlu_without_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition & ~local_irAlu_competing_conditions

# conditions for asAlu flanked by local competing irAlu
ssAlu_flanked_by_irAlu_with_local_competition = (irAlu_pre_condition1 | irAlu_pre_condition2) & ssAlu_condition & local_irAlu_competing_conditions

# First verify the conditions
total_ssAlu = ssAlu_condition.sum()
sum_subtypes = (ssAlu_not_flanked_by_irAlu_condition | 
               ssAlu_flanked_by_irAlu_without_local_competition |
               ssAlu_flanked_by_irAlu_with_local_competition).sum()

condition1_met = (sum_subtypes == total_ssAlu)
if condition1_met:
    print("Great! Sum of three ssAlu subtypes equal to total ssAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of three ssAlu subtypes does not equal to total ssAlu")

# Check for overlaps between subtypes
overlap1 = (ssAlu_not_flanked_by_irAlu_condition & 
           ssAlu_flanked_by_irAlu_without_local_competition).sum()
overlap2 = (ssAlu_not_flanked_by_irAlu_condition & 
           ssAlu_flanked_by_irAlu_with_local_competition).sum()
overlap3 = (ssAlu_flanked_by_irAlu_without_local_competition & 
           ssAlu_flanked_by_irAlu_with_local_competition).sum()
condition2_met = (overlap1 + overlap2 + overlap3) == 0
if condition2_met:
    print(f"Great! No overlapping samples between ssAlu subtypes")
else:
    print(f"Oh no! Overlapping samples between ssAlu subtypes are found")

########################################################################
"""Now let's set up conditions to break irAlu type into 2 subtypes"""
########################################################################
"""
    (1) irAlu without internal competing irAlu: "irAlu_without_intronic_irAlu_competition"
        |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                     <=                   =>     (irAlu_no_competition1)
      or             =>                   <=     (irAlu_no_competition2)
    ----------------------------------------------------------    
    (2) irAlu with internal competing irAlu   
        |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|                
    or             <=  =>                 <=            
    or             =>  <=                 <=   
    or             <=  =>                 =>            
    or             =>  <=                 =>    
    or               =>                 <=  =>     
    or               =>                 =>  <=     
    or               <=                 <=  =>     
    or               <=                 =>  <=    
"""
# For (1) 
irAlu_no_competition1 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
irAlu_no_competition2 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
irAlu_without_intronic_irAlu_competition = irAlu_no_competition1 | irAlu_no_competition2
# For (2)
ui_irAlu_condition = (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_ssAlu_ui'] > 0.0) 
di_irAlu_condition = (cass_df['all_asAlu_di'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0) 
irAlu_no_competition1 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & ui_irAlu_condition & ((cass_df['all_ssAlu_di'] > 0.0) | (cass_df['all_asAlu_di'] > 0.0))
irAlu_no_competition2 = (cass_df['all_asAlu_exon'] == 0.0) & (cass_df['all_ssAlu_exon'] == 0.0) & di_irAlu_condition & ((cass_df['all_ssAlu_ui'] > 0.0) | (cass_df['all_asAlu_ui'] > 0.0))
irAlu_with_intronic_irAlu_competition = irAlu_no_competition1 | irAlu_no_competition2 

# First verify the conditions
total_asAlu = irAlu_condition.sum()
sum_subtypes = (irAlu_without_intronic_irAlu_competition | 
               irAlu_with_intronic_irAlu_competition).sum()

condition1_met = (sum_subtypes == total_asAlu)
if condition1_met:
    print("Great! Sum of two irAlu subtypes equal to total irAlu")
else:
    print("Oh No! There's non-resolved condition. Sum of two irAlu subtypes does not equal to total irAlu")

# Check for overlaps between subtypes
overlap = (irAlu_without_intronic_irAlu_competition & 
           irAlu_with_intronic_irAlu_competition).sum()
if overlap == 0:
    print(f"Great! No overlapping samples between subtypes")
else:
    print(f"Oh no! Overlapping samples between subtypes are found")

########################################################################
"""Now let's set up conditions to break other samples type into 2 subtypes"""
########################################################################
""" (1) other samples contain at least one Alu element in each of two flanking introns and in same orientations
    |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                 <=                    <=   
      or         =>                    => 
    (2) other samples contain at least one Alu element in only one flanking intron
    |Exon1|-----(ui)-----|Exon2|-----(di)-----|Exon3|
                <=                                  (other_with_asAlu_only_ui)
      or        =>                                  (other_with_ssAlu_only_ui)
      or                              <=            (other_with_asAlu_only_di)       
      or                              =>            (other_with_ssAlu_only_di)
      or       <= =>                                (other_with_irAlu_only_ui)
      or                             <= =>          (other_with_irAlu_only_di)
    (3) remaining other samples
    """
other_samples_with_alu_condition = other_samples_condition & (
    (cass_df['all_asAlu_ui'] > 0.0) |
    (cass_df['all_ssAlu_ui'] > 0.0) |
    (cass_df['all_asAlu_di'] > 0.0) |
    (cass_df['all_ssAlu_di'] > 0.0)
)

other_samples_with_alus_in_two_introns_condition = other_samples_condition & (
    ((cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0)) |
    ((cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0))
)

other_samples_with_alus_in_one_intron_condition = other_samples_with_alu_condition & ~other_samples_with_alus_in_two_introns_condition

other_samples_without_alu_condition = other_samples_condition & ~other_samples_with_alu_condition

combined = (
    other_samples_with_alus_in_two_introns_condition |
    other_samples_with_alus_in_one_intron_condition |
    other_samples_without_alu_condition
)
other_sample_condition_met = combined.equals(other_samples_condition)

if other_sample_condition_met:
    print("Great! Sum of two other samples subtypes equal to total other samples")
else:
    print("Oh No! There's non-resolved condition. Sum of two other samples subtypes does not equal to total asAlu")

other_sample_overlap1 = ((other_samples_with_alus_in_two_introns_condition & other_samples_with_alus_in_one_intron_condition).sum() == 0)
other_sample_overlap2 = ((other_samples_with_alus_in_two_introns_condition & other_samples_without_alu_condition).sum() == 0)
other_sample_overlap3 = ((other_samples_with_alus_in_one_intron_condition & other_samples_without_alu_condition).sum() == 0)
other_sample_on_overlap_condition_met = (
    other_sample_overlap1 and other_sample_overlap2 and other_sample_overlap3
)

if other_sample_on_overlap_condition_met:
    print(f"Great! No overlapping samples between subtypes")
else:
    print(f"Oh no! Overlapping samples between subtypes are found")

Great! Sum of three asAlu subtypes equal to total asAlu
Great! No overlapping samples between asAlu subtypes
Great! Sum of three ssAlu subtypes equal to total ssAlu
Great! No overlapping samples between ssAlu subtypes
Great! Sum of two irAlu subtypes equal to total irAlu
Great! No overlapping samples between subtypes
Great! Sum of two other samples subtypes equal to total other samples
Great! No overlapping samples between subtypes


In [14]:
# If the two condition checks are satified, proceed annotating alu subtypes for asAlu
conditions = [
    other_samples_with_alus_in_one_intron_condition,
    other_samples_with_alus_in_two_introns_condition,
    other_samples_without_alu_condition,
    asAlu_not_flanked_by_irAlu_condition,
    asAlu_flanked_by_irAlu_without_local_competition,
    asAlu_flanked_by_irAlu_with_local_competition,
    ssAlu_not_flanked_by_irAlu_condition,
    ssAlu_flanked_by_irAlu_without_local_competition,
    ssAlu_flanked_by_irAlu_with_local_competition,
    irAlu_without_intronic_irAlu_competition,
    irAlu_with_intronic_irAlu_competition
]
    
choices = [
    'other_with_Alu_in_one_flanking_intron',
    'other_with_same_strand_Alu_in_two_flanking_introns',
    'other_without_Alu',
    'asAlu_no_flanking_irAlu',
    'asAlu_with_flanking_noncompete_irAlu',
    'asAlu_with_flanking_compete_irAlu',
    'ssAlu_no_flanking_irAlu',
    'ssAlu_with_flanking_noncompete_irAlu',
    'ssAlu_with_flanking_compete_irAlu',
    'irAlu_no_intronic_irAlu_compete',
    'irAlu_with_intronic_irAlu_compete'
]
    
cass_df.insert(
    loc=cass_df.columns.get_loc('alu_type') + 1,
    column='alu_subtype',
    value=np.select(conditions, choices, default='other')
)

# Verify results
print("\nSubtype distribution:")
print(cass_df['alu_subtype'].value_counts())


Subtype distribution:
alu_subtype
irAlu_with_intronic_irAlu_compete                     13770
other_with_Alu_in_one_flanking_intron                 13632
other_without_Alu                                      9816
irAlu_no_intronic_irAlu_compete                        1221
other_with_same_strand_Alu_in_two_flanking_introns     1204
asAlu_with_flanking_compete_irAlu                      1091
asAlu_with_flanking_noncompete_irAlu                    984
asAlu_no_flanking_irAlu                                 603
ssAlu_no_flanking_irAlu                                 179
ssAlu_with_flanking_noncompete_irAlu                    136
ssAlu_with_flanking_compete_irAlu                       125
Name: count, dtype: int64


### Add a column "alu_specific_subtype" to describe each exon event which further break down the "Other" subtype

In [15]:
# Base unchanged types
unchanged_subtypes = [
    'ssAlu',
    'other_without_Alu',
    'asAlu_no_flanking_irAlu',
    'asAlu_with_flanking_noncompete_irAlu',
    'asAlu_with_flanking_compete_irAlu',
    'irAlu_no_intronic_irAlu_compete',
    'irAlu_with_intronic_irAlu_compete'
]

# Start with the existing 'alu_subtype' column
cass_df.insert(
    loc=cass_df.columns.get_loc('alu_subtype') + 1,
    column='alu_specific_subtype',
    value=cass_df['alu_subtype']
)

# === Split subtype: 'other_with_Alu_in_one_flanking_intron' ===

# Define sub-conditions
asAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0) &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
)

asAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0) &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] == 0.0)
)

ssAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

ssAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

irAlu_only_ui = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] == 0.0) &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] == 0.0)
)

irAlu_only_di = (
    (cass_df['alu_subtype'] == 'other_with_Alu_in_one_flanking_intron') &
    (cass_df['all_ssAlu_ui'] == 0.0) & (cass_df['all_ssAlu_di'] > 0.0) &
    (cass_df['all_asAlu_ui'] == 0.0) & (cass_df['all_asAlu_di'] > 0.0)
)

# Apply annotations
cass_df.loc[asAlu_only_ui, 'alu_specific_subtype'] = 'other_with_asAlu_only_ui'
cass_df.loc[asAlu_only_di, 'alu_specific_subtype'] = 'other_with_asAlu_only_di'
cass_df.loc[ssAlu_only_ui, 'alu_specific_subtype'] = 'other_with_ssAlu_only_ui'
cass_df.loc[ssAlu_only_di, 'alu_specific_subtype'] = 'other_with_ssAlu_only_di'
cass_df.loc[irAlu_only_ui, 'alu_specific_subtype'] = 'other_with_irAlu_only_ui'
cass_df.loc[irAlu_only_di, 'alu_specific_subtype'] = 'other_with_irAlu_only_di'

# === Split subtype: 'other_with_same_strand_Alu_in_two_flanking_introns' ===

asAlu_both = (
    (cass_df['alu_subtype'] == 'other_with_same_strand_Alu_in_two_flanking_introns') &
    (cass_df['all_asAlu_ui'] > 0.0) & (cass_df['all_asAlu_di'] > 0.0)
)

ssAlu_both = (
    (cass_df['alu_subtype'] == 'other_with_same_strand_Alu_in_two_flanking_introns') &
    (cass_df['all_ssAlu_ui'] > 0.0) & (cass_df['all_ssAlu_di'] > 0.0)
)

cass_df.loc[asAlu_both, 'alu_specific_subtype'] = 'other_with_asAlu_in_both_ui_di'
cass_df.loc[ssAlu_both, 'alu_specific_subtype'] = 'other_with_ssAlu_in_both_ui_di'

cass_df['alu_specific_subtype'].value_counts()



alu_specific_subtype
irAlu_with_intronic_irAlu_compete       13770
other_without_Alu                        9816
other_with_irAlu_only_di                 3531
other_with_irAlu_only_ui                 3287
other_with_asAlu_only_di                 1771
other_with_ssAlu_only_di                 1768
other_with_asAlu_only_ui                 1706
other_with_ssAlu_only_ui                 1569
irAlu_no_intronic_irAlu_compete          1221
asAlu_with_flanking_compete_irAlu        1091
asAlu_with_flanking_noncompete_irAlu      984
other_with_asAlu_in_both_ui_di            678
asAlu_no_flanking_irAlu                   603
other_with_ssAlu_in_both_ui_di            526
ssAlu_no_flanking_irAlu                   179
ssAlu_with_flanking_noncompete_irAlu      136
ssAlu_with_flanking_compete_irAlu         125
Name: count, dtype: int64

### Add a column "ku_tag_summary" to summarize Ku tag numbers in ui, exon, di respectively

In [16]:
""" Insert Ku_tag_summary column to summarize ku tag numebers"""
cass_df['ku_tag_summary'] = (
    'asUI=' + cass_df['Ku_asAlu_ui'].astype(str) + ', ' +
    'ssUI=' + cass_df['Ku_ssAlu_ui'].astype(str) + ', ' +
    'asExon=' + cass_df['Ku_asAlu_exon'].astype(str) + ', ' +
    'ssExon=' + cass_df['Ku_ssAlu_exon'].astype(str) + ', ' +
    'asDI=' + cass_df['Ku_asAlu_di'].astype(str) + ', ' +
    'ssDI=' + cass_df['Ku_ssAlu_di'].astype(str)
)

cass_df.insert(
    loc=cass_df.columns.get_loc('alu_specific_subtype') + 1,
    column='ku_tag_summary',
    value=cass_df.pop('ku_tag_summary')  # remove & re-insert
)

""" Insert Ku_bound column to summarize whether a cassette exon has Ku tags bind to it
    --for asAlus, Ku_bound = 1 if ku_asAlu_exon > 0.0
    --for ssAlus, Ku_bound = 1 if ku_ssAlu_exon > 0.0
    --for irAlus and other, Ku_bound = 1 if any of [ku_asAlu_ui,ku_asAlu_di,ku_ssAlu_ui,ku_ssAlu_di] > 0.0
    """
cass_df['ku_bound'] = 0

# Apply rules based on alu_type
cass_df.loc[
    (cass_df['alu_type'] == 'asAlu') & (cass_df['Ku_asAlu_exon'] > 0.0),
    'ku_bound'
] = 1

cass_df.loc[
    (cass_df['alu_type'] == 'ssAlu') & (cass_df['Ku_ssAlu_exon'] > 0.0),
    'ku_bound'
] = 1

cass_df.loc[
    (cass_df['alu_type'].isin(['irAlu', 'other'])) &
    (
        (cass_df['Ku_asAlu_ui'] > 0.0) |
        (cass_df['Ku_asAlu_di'] > 0.0) |
        (cass_df['Ku_ssAlu_ui'] > 0.0) |
        (cass_df['Ku_ssAlu_di'] > 0.0)
    ),
    'ku_bound'
] = 1

# Step 4: Insert ku_bound after ku_tag_summary
cass_df.insert(
    loc=cass_df.columns.get_loc('ku_tag_summary') + 1,
    column='ku_bound',
    value=cass_df.pop('ku_bound')
)

In [17]:
cass_df['ku_bound'].value_counts()

ku_bound
0    21800
1    20961
Name: count, dtype: int64

### Add a column '100vs0_10vs0_Overlap' to indicate whether a DSE belongs to 100vs0 or 10vs0 or both conditions. Also add a column 'sensitivity' to categorize DSEs

In [18]:
# Define filter condition
filter_condition = (
    cass_df['filter(HCT116.100%Ku_vs_0%Ku)'].isin([-1, 1]) |
    cass_df['filter(HCT116.10%Ku_vs_0%Ku)'].isin([-1, 1])
)

# Define opposite direction condition
opposite_direction_condition = (
    (cass_df['dI_g1_vs_g2(HCT116.10%Ku_vs_0%Ku)'] > 0) & 
    (cass_df['dI_g1_vs_g2(HCT116.100%Ku_vs_0%Ku)'] < 0)
) | (
    (cass_df['dI_g1_vs_g2(HCT116.10%Ku_vs_0%Ku)'] < 0) & 
    (cass_df['dI_g1_vs_g2(HCT116.100%Ku_vs_0%Ku)'] > 0)
)

# Determine overlap types
is_100 = cass_df['filter(HCT116.100%Ku_vs_0%Ku)'].isin([-1, 1])
is_10 = cass_df['filter(HCT116.10%Ku_vs_0%Ku)'].isin([-1, 1])

overlap_annotation = pd.Series('', index=cass_df.index)
overlap_annotation[is_100 & ~is_10] = '100% vs 0% only'
overlap_annotation[is_10 & ~is_100] = '10% vs 0% only'
overlap_annotation[is_100 & is_10] = 'common'

# Insert 100vs0_10vs0_Overlap column
cass_df.insert(
    cass_df.columns.get_loc('alu_subtype') + 1,
    '100vs0_10vs0_Overlap',
    overlap_annotation
)

# Initialize sensitivity annotation
sensitivity = pd.Series('', index=cass_df.index)

# Apply sensitivity annotation rules
common = cass_df['100vs0_10vs0_Overlap'] == 'common'
opposite = opposite_direction_condition

sensitivity[filter_condition & common] = 'high'
sensitivity[filter_condition & common & opposite] = 'outlier'
sensitivity[filter_condition & ~common & opposite] = 'low'
sensitivity[filter_condition & ~common & ~opposite] = 'middle'

# Insert sensitivity column
cass_df.insert(
    cass_df.columns.get_loc('100vs0_10vs0_Overlap') + 1,
    'sensitivity',
    sensitivity
)


In [19]:
cass_df[cass_df['alu_type'].isin(['asAlu', 'irAlu', 'ssAlu'])]['sensitivity'].value_counts()

sensitivity
          16950
middle      556
high        481
low         122
Name: count, dtype: int64

In [20]:
cass_df['sensitivity'].value_counts()

sensitivity
           40124
middle      1388
high         967
low          281
outlier        1
Name: count, dtype: int64

In [21]:
cass_df[cass_df['alu_type'].isin(['asAlu', 'irAlu', 'ssAlu'])]['100vs0_10vs0_Overlap'].value_counts()

100vs0_10vs0_Overlap
                   16950
common               481
100% vs 0% only      413
10% vs 0% only       265
Name: count, dtype: int64

### Add a column "exon_sizes" to describe each length of each exon

In [27]:
cass_df_copy = cass_df.copy()

bed_columns = [
    'chrom', 'start', 'end', 'name', 'score', 'strand',
    'thick_start', 'thick_end', 'rgb', 'block_count', 'exon_sizes', 'block_starts'
]
# Read the BED file
cass_anno_bed_file_path = "/Users/tianji/Desktop/Alu Project/CLIP Data/Alu-CassExon Reference Data/Hs.seq.all.cass.chrom.can.bed"
cass_anno = pd.read_csv(cass_anno_bed_file_path, sep='\t', header=None, names=bed_columns)

# Merge cass_anno's 'exon_sizes' into cass_df after 'chromLength'
cass_df = pd.merge(
    cass_df_copy,
    cass_anno[['name', 'exon_sizes']],  # Select only 'name' and 'exon_sizes' from cass_anno
    on='name',                           # Merge on the 'name' column
    how='left'                           # Keep all rows from cass_df even if no match
)

# Reorder columns to place 'exon_sizes' after 'chromLength'
cols = cass_df.columns.tolist()
chromLength_idx = cols.index('chromLength')
cols.insert(chromLength_idx + 1, cols.pop(cols.index('exon_sizes')))  # Move 'exon_sizes' after 'chromLength'
cass_df = cass_df[cols]

### Add columns "all_asAlu_type", "all_asAlu_id" and "all_asAlu_family" to indicate what asAlu elements overlap with the middle exon in each cassette exon event

In [31]:
asAlu2exon = pd.read_csv("/Users/tianji/Desktop/Alu Project/Alu_Family_Analysis/asAlu.Overlap.CassMiddleExon.csv")
cass_df_copy = cass_df.copy()

# Step 1: Rename Alu-related columns in asAlu2exon to avoid conflicts with column names in cass_df_copy before merge
asAlu2exon = asAlu2exon.rename(columns={
    'Alu_type': 'all_asAlu_type',
    'Alu_id': 'all_asAlu_id',
    'Alu_family': 'all_asAlu_family',
    'ku_regulated': 'all_asAlu_Ku_regulated'
})

# Step 2: Merge with cass_df_copy on 'name'
cass_df_copy = cass_df_copy.merge(
    asAlu2exon[['name', 'all_asAlu_type', 'all_asAlu_id', 'all_asAlu_family', 'all_asAlu_Ku_regulated']],
    on='name',
    how='left'
)

# Step 3: Insert 'all_asAlu_type', 'all_asAlu_id', 'all_asAlu_family', and 'all_asAlu_Ku_regulated' after the 'all_asAlu_exon' column in cass_df_copy
new_cols = ['all_asAlu_type', 'all_asAlu_id', 'all_asAlu_family', 'all_asAlu_Ku_regulated']

# Find the insertion index just after 'all_asAlu_exon'
cols = list(cass_df_copy.columns)
insert_at = cols.index('all_asAlu_exon') + 1

# Pop & insert each new column in the right order
for col in new_cols:
    # .pop(col) removes the column and returns its Series
    cass_df_copy.insert(insert_at, col, cass_df_copy.pop(col))
    insert_at += 1


In [32]:
cass_df = cass_df_copy

### Perform t-test between Gtex psi in Brain Regions and Other Regions. Save results into new columns

In [33]:
alpha = 0.05  # significance threshold

# Get Gtex psi from all regions
Gtex_df = cass_df.loc[:,'Adipose-Subcutaneous':'Vagina'].copy()

# Identify groups
brain_region_cols = Gtex_df.loc[:, 'Brain-Amygdala':'Brain-Substantia_nigra'].columns
print(f"There are a total of {len(brain_region_cols)} brain region columns")
#print(f"Brain region columns are: {brain_region_cols}")
other_region_cols = Gtex_df.columns.difference(brain_region_cols)
print(f"There are a total of {len(other_region_cols)} other region columns")
#print(f"Other region columns are: {other_region_cols}")
total_cols = len(brain_region_cols) + len(other_region_cols)

# Lists to store t-test results
mw_pvals = []
mw_significance = []
ttest_pvals = []
ttest_significance = []
num_samples_filtered_out = 0

##########################################################
####################Calculate p-values####################
##########################################################

for idx, row in tqdm(Gtex_df.iterrows(), total=len(Gtex_df), desc="Processing samples"):
    # Skip if a sample misses more than 10% of the Gtex Data
    total_columns_per_samples = row.count()
    if total_columns_per_samples < 0.9 * total_cols:
        mw_pvals.append(np.nan)
        mw_significance.append(0)
        ttest_pvals.append(np.nan)
        ttest_significance.append(0)
        num_samples_filtered_out += 1
        continue
    
    # Extract values from each group
    brain_region_data = row[brain_region_cols].dropna()
    other_region_data = row[other_region_cols].dropna()  # Fixed typo in variable name
    
    # Mann–Whitney U test
    if len(brain_region_data) >= 2 and len(other_region_data) >= 2:
        stat_mw, pval_mw = mannwhitneyu(brain_region_data, other_region_data, alternative='two-sided')
    else:
        pval_mw = np.nan
    mw_pvals.append(pval_mw)
    mw_significance.append(1 if (not np.isnan(pval_mw) and pval_mw < alpha) else 0)

    # Standard t-test (Welch's t-test assuming unequal variance)
    if len(brain_region_data) >= 2 and len(other_region_data) >= 2:
        stat_t, pval_t = ttest_ind(brain_region_data, other_region_data, equal_var=False)
    else:
        pval_t = np.nan
    ttest_pvals.append(pval_t)
    ttest_significance.append(1 if (not np.isnan(pval_t) and pval_t < alpha) else 0)

#####################################################################################################
####################Calculate adjust p-values with Benjamini-Hochberg (BH) method####################
#####################################################################################################

def bh_fdr_adjustment(pvals_with_nan):
    """Apply BH FDR correction while preserving NaN positions"""
    # Get valid p-values and their original indices
    valid_mask = ~np.isnan(pvals_with_nan)
    valid_pvals = np.array(pvals_with_nan)[valid_mask]
    
    # Perform BH correction
    _, fdr_pvals = multipletests(valid_pvals, alpha=alpha, method='fdr_bh')[:2]
    
    # Reconstruct full array with NaNs
    adjusted = np.full_like(pvals_with_nan, np.nan, dtype=float)
    adjusted[valid_mask] = fdr_pvals
    return adjusted.tolist()

# Calculate FDR-adjusted p-values for both tests
mw_fdr = bh_fdr_adjustment(mw_pvals)
ttest_fdr = bh_fdr_adjustment(ttest_pvals)

# Update significance flags using FDR-adjusted values
mw_significance_fdr = [1 if (not np.isnan(p) and p < alpha) else 0 for p in mw_fdr]
ttest_significance_fdr = [1 if (not np.isnan(p) and p < alpha) else 0 for p in ttest_fdr]

# Print summary
print(f"\nAfter BH-FDR correction:")
print(f"Mann-Whitney: {sum(mw_significance_fdr)} significant at FDR < {alpha}")
print(f"Welch's t-test: {sum(ttest_significance_fdr)} significant at FDR < {alpha}")
print(f"Samples (have more than 10% of Gtex psi missing) filtered out: {num_samples_filtered_out}")

There are a total of 13 brain region columns
There are a total of 41 other region columns


Processing samples:   1%|          | 404/42761 [00:00<00:20, 2019.10it/s]/Users/tianji/mambaforge/envs/myenv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
Processing samples: 100%|██████████| 42761/42761 [00:21<00:00, 1981.97it/s]



After BH-FDR correction:
Mann-Whitney: 17458 significant at FDR < 0.05
Welch's t-test: 16626 significant at FDR < 0.05
Samples (have more than 10% of Gtex psi missing) filtered out: 16998


In [34]:
# Insert t-test results into four columns
# 1. Find the column index of 'Adipose-Subcutaneous'
loc_adipose_subcut = cass_df.columns.get_loc('Adipose-Subcutaneous')

# 2. Insert columns at that index (the new columns will appear right BEFORE 'Adipose-Subcutaneous')
cass_df.insert(loc_adipose_subcut, 'pvalue(mannwhitney)', mw_pvals)
cass_df.insert(loc_adipose_subcut + 1, 'FDR(mannwhitney)', mw_fdr)
cass_df.insert(loc_adipose_subcut + 2, 'significance(mannwhitney)', mw_significance_fdr)
cass_df.insert(loc_adipose_subcut + 3, 'pvalue(ttest)', ttest_pvals)
cass_df.insert(loc_adipose_subcut + 4, 'FDR(ttest)', ttest_fdr)
cass_df.insert(loc_adipose_subcut + 5, 'significance(ttest)', ttest_significance_fdr)

### Insert in_frame column to indicate if the middle exon is dividable by 3

In [35]:
# Assuming cass_df is your DataFrame

# 1. Extract the second substring from exon_sizes
cass_df['exon_len2'] = cass_df['exon_sizes'].str.split(',').str[1]

# 2. Convert to integer, handling errors
cass_df['exon_len2'] = pd.to_numeric(cass_df['exon_len2'], errors='coerce')

# 3. Create in_frame column based on divisibility by 3
cass_df['in_frame'] = np.where(
    cass_df['exon_len2'] % 3 == 0, 
    'in', 
    np.where(cass_df['exon_len2'].isna(), np.nan, 'out')
)

# 4. Find position of exon_sizes column
exon_sizes_idx = cass_df.columns.get_loc('exon_sizes')

# 5. Insert in_frame column after exon_sizes
cols = cass_df.columns.tolist()
cols.insert(exon_sizes_idx + 1, cols.pop(cols.index('in_frame')))
cass_df = cass_df[cols]

# Optional: Remove temporary exon_len2 column
cass_df = cass_df.drop(columns=['exon_len2'])

# Verify the result
print(cass_df[['exon_sizes', 'in_frame']].head())

    exon_sizes in_frame
0   297,87,273       in
1  128,116,144      out
2   102,72,128       in
3   130,65,102      out
4   88,215,129      out


### Save all the merged data frames

In [27]:
# Save each DataFrame as a separate CSV file
cass_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.Merged.DiffAlterSplicing.results.step1.csv", index=False)
alt3_merged_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Alt3.Merged.DiffAlterSplicing.results.csv", index=False)
alt5_merged_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Alt5.Merged.DiffAlterSplicing.results.csv", index=False)
iret_merged_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Iret.Merged.DiffAlterSplicing.results.csv", index=False)
mutx_merged_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Mutx.Merged.DiffAlterSplicing.results.csv", index=False)
taca_merged_df.to_csv("/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Taca.Merged.DiffAlterSplicing.results.csv", index=False)

# Delete

In [36]:
cass_as_result = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.Merged.DiffAlterSplicing.results.step1.csv')
cass_new_aluoverlap = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.071825.csv')
cass_controls = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.Ctrl.081325.csv')
cols_to_remove = [
    'alu_type', 'alu_subtype', 'alu_specific_subtype',
    'ku_tag_summary', 'ku_bound'
]

# Drop the named ones + the last 34 columns
cass_as_result = cass_as_result.drop(columns=cols_to_remove + list(cass_as_result.columns[-34:]))

# Select columns from cass_new_aluoverlap[213:] and merge
cols_to_merge = cass_new_aluoverlap.columns[213:]
merged = cass_as_result.merge(
    cass_new_aluoverlap[['name'] + list(cols_to_merge)],
    on='name',
    how='inner'  # ensures only common names are kept
)

# Select the last 71 columns from cass_controls
cols_to_add = cass_controls.columns[-71:]
merged = merged.merge(
    cass_controls[['name'] + list(cols_to_add)],
    on='name',
    how='inner'
)

/var/folders/gp/l_62hzx91kz0hng_60kps5t00000gn/T/ipykernel_62732/1496339259.py:2: DtypeWarning: Columns (230) have mixed types. Specify dtype option on import or set low_memory=False.
  cass_new_aluoverlap = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.071825.csv')
/var/folders/gp/l_62hzx91kz0hng_60kps5t00000gn/T/ipykernel_62732/1496339259.py:3: DtypeWarning: Columns (232) have mixed types. Specify dtype option on import or set low_memory=False.
  cass_controls = pd.read_csv('/Users/tianji/Desktop/Alu Project/Output/Merged_DSE_Results/Cass.AluOverlap.Ctrl.081325.csv')


In [42]:
merged.to_csv('/Users/tianji/Desktop/Alu Project/100vs0&10vs0 Output/Cass.Merged.Results.csv')

In [ ]:
# To adjust the exon_sizes column
#merged.to_csv('/Users/tianji/Desktop/Alu Project/100vs0&10vs0 Output/Cass.Merged.Results.txt', index=False, sep="\t")